In [2]:
#!/usr/bin/env python

import argparse
import os
import sys
from glob import glob
import six
import pickle

import pandas as pd
import keras as K

from sklearn.model_selection import ShuffleSplit

import pescador
import librosa
from jams.util import smkdirs

import pumpp
import random
import h5py
import jams
import numpy as np
import json

seed = random.seed(20180319)

In [3]:
def process_arguments(args):
    parser = argparse.ArgumentParser(description=__doc__)

    parser.add_argument('--max_samples', dest='max_samples', type=int,
                        default=128,
                        help='Maximum number of samples to draw per streamer')

    parser.add_argument('--patch-duration', dest='duration', type=float,
                        default=8.0,
                        help='Duration (in seconds) of training patches')

    parser.add_argument('--seed', dest='seed', type=int,
                        default='20170412',
                        help='Seed for the random number generator')

    parser.add_argument('--train-streamers', dest='train_streamers', type=int,
                        default=1024,
                        help='Number of active training streamers')

    parser.add_argument('--batch-size', dest='batch_size', type=int,
                        default=32,
                        help='Size of training batches')

    parser.add_argument('--rate', dest='rate', type=int,
                        default=8,
                        help='Rate of pescador stream deactivation')

    parser.add_argument('--epochs', dest='epochs', type=int,
                        default=100,
                        help='Maximum number of epochs to train for')

    parser.add_argument('--epoch-size', dest='epoch_size', type=int,
                        default=512,
                        help='Number of batches per epoch')

    parser.add_argument('--validation-size', dest='validation_size', type=int,
                        default=1024,
                        help='Number of batches per validation')

    parser.add_argument('--early-stopping', dest='early_stopping', type=int,
                        default=20,
                        help='# epochs without improvement to stop')

    parser.add_argument('--reduce-lr', dest='reduce_lr', type=int,
                        default=10,
                        help='# epochs before reducing learning rate')

    parser.add_argument(dest='working', type=str,
                        help='Path to working directory')

    return parser.parse_args(args)

In [4]:
OUTPUT_PATH = '/scratch/yw3004/projects/deepunet/'

## Make sampler

In [5]:
def make_sampler(max_samples, duration, pump, seed):

    n_frames = librosa.time_to_frames(duration,
                                      sr=pump['stft'].sr,
                                      hop_length=pump['stft'].hop_length)
    
    # Only make this object when building the sampler (during training)
    fake_p_stft = pumpp.feature.STFTMag(name='output', sr=pump['stft'].sr, 
                                        hop_length=pump['stft'].hop_length, n_fft=pump['stft'].n_fft, 
                                        log=False, conv='tf')
    
    # Make a sampler using real transformer and the fake one for output
    return pumpp.Sampler(max_samples, n_frames, pump['stft'], fake_p_stft, random_state=seed)

In [6]:
def load_h5(fname):

    data = {}

    def collect(k, v):
        if isinstance(v, h5py.Dataset):
            data[k] = v.value

    with h5py.File(fname, mode='r') as hf:
        hf.visititems(collect)

    return data

## Original data generation functions (don't work)

In [9]:
def data_sampler(fname, sampler):
    #'''Generate samples from a specified h5 file'''
    for datum in sampler(load_h5(fname)):
        yield datum

In [10]:
def data_generator(working, tracks, sampler, n_active, rate, batch_size=32,
                   **kwargs):
    #'''Generate a data stream from a collection of tracks and a sampler'''

    streams = []

    for track in tracks:
        fname = os.path.join(working,
                             os.path.extsep.join([str(track), '.h5']))
        streams.append(pescador.Streamer(data_sampler, fname, sampler))

    # Send it all to a mux
    mux = pescador.StochasticMux(streams, n_active, rate, **kwargs)
  
    if batch_size == 1:
        return mux
    else:
        return pescador.maps.buffer_stream(mux, batch_size)

## Modified data generation functions (use these)

In [7]:
def data_sampler_2(fname, max_samples, duration, pump, seed ):
    #'''Generate samples from a specified h5 file'''
    sampler = make_sampler(max_samples, duration, pump, seed)

    for datum in sampler(load_h5(fname)):
        #print(fname)
        #print(duration)
        
        #remove the first dimension of arrays 
        datum['stft/mag'] = np.squeeze(datum['stft/mag'], axis=(0,))
        datum['output/mag'] = np.squeeze(datum['output/mag'], axis=(0,))
        yield datum

In [8]:
def data_generator_2(working, tracks, max_samples, duration, pump, seed, n_active, rate, batch_size=32,
                   **kwargs):
    #'''Generate a data stream from a collection of tracks and a sampler'''

    streams = []

    for track in tracks:
        fname = os.path.join(working,
                             os.path.extsep.join([str(track), 'h5']))
        streams.append(pescador.Streamer(data_sampler_2, fname, max_samples, duration, pump, seed))

    # Send it all to a mux
    mux = pescador.StochasticMux(streams, n_active, rate, **kwargs)

    if batch_size == 1:
        return mux
    else:
        return pescador.buffer_stream(mux, batch_size)

## Create training index json files (run ones)

In [30]:
inds = jams.util.find_with_extension('/scratch/yw3004/projects/deepunet/pump/000/', 'h5')
index_train = {}
index_train['id'] = {}
iteration = 0
for ind in inds:
    index_train['id'][iteration] = os.path.basename(ind)[:6]
    iteration += 1

with open('index_train.json', 'w') as fp:
    json.dump(index_train, fp)

## Build the model

In [9]:
def construct_model(pump):

    model_inputs = 'stft/mag'

    # Build the input layer
    x = pump.layers()[model_inputs]
    x_cropped = K.layers.Cropping2D(cropping=((0, 0), (1, 0)), data_format='channels_last')(x)
    print(x._keras_shape)

    #Apply batch normalization  #torch model does not have this layer
    x_bn = K.layers.BatchNormalization()(x_cropped)

    # First convolutional layer: 16 5x5
    conv1 = K.layers.Convolution2D(16, (5, 5), strides=2, 
                                   padding='same',
                                   activation='linear', 
                                   data_format='channels_last')(x_bn)
    batch1 = K.layers.BatchNormalization()(conv1) #no batch normalize in first layer, don't know why
    lrelu1 = K.layers.LeakyReLU(alpha=0.2)(batch1)
    print(lrelu1._keras_shape)
    
    # Second convolutional layer:
    conv2 = K.layers.Convolution2D(32, (5, 5), strides=2, 
                                   padding='same', 
                                   activation='linear',
                                   data_format='channels_last')(lrelu1)
    batch2 = K.layers.BatchNormalization()(conv2)
    lrelu2 = K.layers.LeakyReLU(alpha=0.2)(batch2)
    print(lrelu2._keras_shape)

    # Third convolutional layer: 
    conv3 = K.layers.Convolution2D(64, (5, 5), strides=2, 
                                   padding='same', 
                                   activation='linear',
                                   data_format='channels_last')(lrelu2)
    batch3 = K.layers.BatchNormalization()(conv3)
    lrelu3 = K.layers.LeakyReLU(alpha=0.2)(batch3)
    print(lrelu3._keras_shape)


    # Fourth convolutional layer: 
    conv4 = K.layers.Convolution2D(128, (5, 5), strides=2, 
                                   padding='same', 
                                   activation='linear',
                                   data_format='channels_last')(lrelu3) 
    batch4 = K.layers.BatchNormalization()(conv4)
    lrelu4 = K.layers.LeakyReLU(alpha=0.2)(batch4)
    print(lrelu4._keras_shape)


    # Fifth convolutional layer: 
    conv5 = K.layers.Convolution2D(256, (5, 5), strides=2, 
                                   padding='same', 
                                   activation='linear',
                                   data_format='channels_last')(lrelu4)
    batch5 = K.layers.BatchNormalization()(conv5)
    lrelu5 = K.layers.LeakyReLU(alpha=0.2)(batch5)
    print(lrelu5._keras_shape)
    
    # Sixth convolutional layer: no batch normalization, regular relu
    conv6 = K.layers.Convolution2D(512, (5, 5), strides=2, 
                                   padding='same', 
                                   activation='relu',
                                   data_format='channels_last')(lrelu5)
    print(conv6._keras_shape)
    
    
    # First deconvolutional layer:
    deconv7 = K.layers.Conv2DTranspose(256, (5, 5), strides=2, 
                                       padding='same', 
                                       activation='linear',
                                       data_format='channels_last')(conv6)
    batch7 = K.layers.BatchNormalization()(deconv7)
    dropout7 = K.layers.Dropout(0.5)(batch7)
    concat7 = K.layers.concatenate([dropout7, conv5], axis=3)
    relu7 = K.layers.Activation('relu')(concat7)
    print(relu7._keras_shape)
    
    # Second deconvolutional layer:
    deconv8 = K.layers.Conv2DTranspose(128, (5, 5), strides=2, 
                                       padding='same', 
                                       activation='linear',
                                       data_format='channels_last')(relu7)
    batch8 = K.layers.BatchNormalization()(deconv8)
    dropout8 = K.layers.Dropout(0.5)(batch8)
    concat8 = K.layers.concatenate([dropout8, conv4], axis=3)
    relu8 = K.layers.Activation('relu')(concat8)
    print(relu8._keras_shape)
    
    # Third deconvolutional layer:
    deconv9 = K.layers.Conv2DTranspose(64, (5, 5), strides=2, 
                                       padding='same', 
                                       activation='linear',
                                       data_format='channels_last')(relu8)
    batch9 = K.layers.BatchNormalization()(deconv9)
    dropout9 = K.layers.Dropout(0.5)(batch9)
    concat9 = K.layers.concatenate([dropout9, conv3], axis=3)
    relu9 = K.layers.Activation('relu')(concat9)
    print(relu9._keras_shape)
    
    # Fourth deconvolutional layer:
    deconv10 = K.layers.Conv2DTranspose(32, (5, 5), strides=2, 
                                       padding='same', 
                                       activation='linear',
                                       data_format='channels_last')(relu9)
    batch10 = K.layers.BatchNormalization()(deconv10)
    concat10 = K.layers.concatenate([batch10, conv2], axis=3)
    relu10 = K.layers.Activation('relu')(concat10)
    print(relu10._keras_shape)
    
    # Fifth deconvolutional layer:
    deconv11 = K.layers.Conv2DTranspose(16, (5, 5), strides=2, 
                                       padding='same', 
                                       activation='linear',
                                       data_format='channels_last')(relu10)
    batch11 = K.layers.BatchNormalization()(deconv11)
    concat11 = K.layers.concatenate([batch11, conv1], axis=3)
    relu11 = K.layers.Activation('relu')(concat11)
    print(relu11._keras_shape)

    # Last deconolutional layer: Mask layer
    deconv12 = K.layers.Conv2DTranspose(1, (5, 5), strides=2, 
                                       padding='same', 
                                       activation='sigmoid',
                                       data_format='channels_last')(relu11)
    print(deconv12._keras_shape)
    
    # Final output: should be pointwise multiplication
    outputs = K.layers.Multiply()([x_cropped, deconv12])
    print(outputs._keras_shape)
    
    padded_outputs = K.layers.ZeroPadding2D(padding=((0, 0),(1, 0)), data_format='channels_last')(outputs)
    print(padded_outputs._keras_shape)

    model = K.models.Model(x, padded_outputs)
    model_outputs = 'output/mag'

    return model, [model_inputs], [model_outputs]

## Training

In [10]:
def train(working, max_samples, duration, rate,
          batch_size, epochs, epoch_size, validation_size,
          early_stopping, reduce_lr, seed):
    '''
    Parameters
    ----------
    working : str
        directory that contains the experiment data (h5)
    max_samples : int
        Maximum number of samples per streamer
    duration : float
        Duration of training patches
    batch_size : int
        Size of batches
    rate : int
        Poisson rate for pescador
    epochs : int
        Maximum number of epoch
    epoch_size : int
        Number of batches per epoch
    validation_size : int
        Number of validation batches
    early_stopping : int
        Number of epochs before early stopping
    reduce_lr : int
        Number of epochs before reducing learning rate
    seed : int
        Random seed
    '''

    # Load the pump
    with open(os.path.join(OUTPUT_PATH, 'pump.pkl'), 'rb') as fd:
        pump = pickle.load(fd)

    # Build the sampler
    sampler = make_sampler(max_samples, duration, pump, seed)

    # Build the model
    model, inputs, outputs = construct_model(pump)

    # Load the training data
    with open('index_train.json', 'r') as fp:
        data = json.load(fp)
        idx_train_ = pd.DataFrame(data)


    # Split the training data into train and validation
    splitter_tv = ShuffleSplit(n_splits=1, test_size=0.25,
                               random_state=seed)
    train, val = next(splitter_tv.split(idx_train_))

    idx_train = idx_train_.iloc[train]
    idx_val = idx_train_.iloc[val]

    gen_train = data_generator_2(working,
                               idx_train['id'].values, max_samples, duration, pump, seed, epoch_size,
                               rate=rate,
                               batch_size=batch_size,
                               #revive=True, #what is this doing?
                               random_state=seed)


    gen_train = pescador.maps.keras_tuples(gen_train, inputs=inputs, outputs=outputs)

    gen_val = data_generator_2(working,
                             idx_val['id'].values, max_samples, duration, pump, seed, len(idx_val),
                             rate=rate,
                             batch_size=batch_size,
                             #revive=True,
                             random_state=seed)


    gen_val = pescador.maps.keras_tuples(gen_val, inputs=inputs, outputs=outputs)

    
    #loss:mean_absolute_error, metric: put loss here
    model.compile(K.optimizers.Adam(), loss='mean_absolute_error')

    # Store the model
    model_spec = K.utils.serialize_keras_object(model)
    with open(os.path.join(OUTPUT_PATH, 'model_spec.pkl'), 'wb') as fd:
        pickle.dump(model_spec, fd)

    # Construct the weight path
    weight_path = os.path.join(OUTPUT_PATH, 'model.h5')

    # Build the callbacks
    #monitor = 'val_loss' #might not need to specify this
    cb = []
    cb.append(K.callbacks.ModelCheckpoint(weight_path,
                                          save_best_only=True,
                                          verbose=1,
                                          ))

    cb.append(K.callbacks.ReduceLROnPlateau(patience=reduce_lr,
                                            verbose=1,
                                            ))

    cb.append(K.callbacks.EarlyStopping(patience=early_stopping,
                                        verbose=1,
                                       ))

    # Fit the model
    model.fit_generator(gen_train, epoch_size, epochs,
                        validation_data=gen_val,
                        validation_steps=validation_size,
                        callbacks=cb)


# if __name__ == '__main__':
#     params = process_arguments(sys.argv[1:])

#     #smkdirs(OUTPUT_PATH)

#     #version = increment_version(os.path.join(OUTPUT_PATH, 'version.txt'))

#     print('{}: training'.format(__doc__))
# #    print('Model version: {}'.format(version))
#     print(params)

#     train(params.working,
#           params.max_samples, params.duration,
#           params.rate,
#           params.batch_size,
#           params.epochs, params.epoch_size,
#           params.validation_size,
#           params.early_stopping,
#           params.reduce_lr,
#           params.seed)

## Training trials

In [40]:
train('/scratch/yw3004/projects/deepunet/pump/000/', 1, 6.0, 8, 32, 100, 512, 10, 20, 10, seed)

(None, None, 513, 1)
(None, None, 256, 16)
(None, None, 128, 32)
(None, None, 64, 64)
(None, None, 32, 128)
(None, None, 16, 256)
(None, None, 8, 512)
(None, None, 16, 512)
(None, None, 32, 256)
(None, None, 64, 128)
(None, None, 128, 64)
(None, None, 256, 32)
(None, None, 512, 1)
(None, None, 512, 1)
(None, None, 513, 1)
Epoch 1/100
512/512 [==============================] - 103s 202ms/step - loss: 0.0048 - val_loss: 0.0048

Epoch 00001: val_loss improved from inf to 0.00475, saving model to /scratch/yw3004/projects/deepunet/model.h5
Epoch 2/100
511/512 [============================>.] - ETA: 0s - loss: 0.0044

ValueError: low >= high